In [1]:
import csv
import math
import datetime

### 用dict保存信息
#user_habit_dict:每个用户的乘车记录:起点,终点,距离
user_habit_dict={}

#start_end_dict:每条记录的起点,终点对
start_end_dict={}

#end_start_dict:每条记录的起点,终点对
end_start_dict={}

#user_habit_dict_test:test中每个用户的记录
user_habit_dict_test={}

#bike_dict:bike中的记录
bike_dict={}

#弧度转换
def rad(tude):
    return (math.pi/180.0)*tude

#geohash模块提取的
__base32 = '0123456789bcdefghjkmnpqrstuvwxyz'
__decodemap = { }
for i in range(len(__base32)):
    __decodemap[__base32[i]] = i
del i

### geohash转经纬度
#返回 精确的经纬度和误差
def decode_exactly(geohash):
    lat_interval, lon_interval = (-90.0, 90.0), (-180.0, 180.0)
    lat_err, lon_err = 90.0, 180.0
    is_even = True
    for c in geohash:
        cd = __decodemap[c]
        for mask in [16, 8, 4, 2, 1]:
            if is_even: # adds longitude info
                lon_err /= 2
                if cd & mask:
                    lon_interval = ((lon_interval[0]+lon_interval[1])/2, lon_interval[1])
                else:
                    lon_interval = (lon_interval[0], (lon_interval[0]+lon_interval[1])/2)
            else:      # adds latitude info
                lat_err /= 2
                if cd & mask:
                    lat_interval = ((lat_interval[0]+lat_interval[1])/2, lat_interval[1])
                else:
                    lat_interval = (lat_interval[0], (lat_interval[0]+lat_interval[1])/2)
            is_even = not is_even
    lat = (lat_interval[0] + lat_interval[1]) / 2
    lon = (lon_interval[0] + lon_interval[1]) / 2
    return lat, lon, lat_err, lon_err

#返回 欧式距离  （其实还可以返回南北方向距离,东西方向距离,曼哈顿距离,方向(-0.5:0.5)，但是删了，没啥吊用）
def produceLocationInfo(latitude1, longitude1,latitude2, longitude2):
    radLat1 = rad(latitude1)
    radLat2 = rad(latitude2)
    a = radLat1-radLat2
    b = rad(longitude1)-rad(longitude2)
    R = 6378137
    d = R*2*math.asin(math.sqrt(math.pow(math.sin(a/2),2)+math.cos(radLat1)*math.cos(radLat2)*math.pow(math.sin(b/2),2)))
    detallat = abs(a)*R
    detalLon = math.sqrt(d**2-detallat**2)
    if b==0:
        direction = 1/2 if a*b>0 else -1/2
    else:
        direction = math.atan(detallat/detalLon*(1 if a*b>0 else -1))/math.pi
    return round(d)

#返回 欧式距离
def loc_2_dis(hotStartLocation,hotEndLocation):
    StartLocation = decode_exactly(hotStartLocation[:7])
    EndLocation = decode_exactly(hotEndLocation[:7])
    latitude1 = StartLocation[0]
    longitude1 = StartLocation[1]
    latitude2 = EndLocation[0]
    longitude2 = EndLocation[1]
    return produceLocationInfo(latitude1, longitude1, latitude2, longitude2)

#返回 是否放假,距0点的分钟数,距5月1的天数
def produceTimeInfo(TimeData):
    TimeData = TimeData.split(' ')
    baseData = datetime.datetime(2017, 5, 1, 0, 0, 1)
    if '-' in TimeData[0]:
        mydata = TimeData[0].split('-')
    else:
        mydata = TimeData[0].split('/')
    mytime = TimeData[1].split(':')
    mydata[0] = int(mydata[0])
    mydata[1] = int(mydata[1])
    mydata[2] = int(mydata[2])
    mytime[0] = int(mytime[0])
    mytime[1] = int(mytime[1])
    #mytime[2] = int(mytime[2].split('.')[0])
    dt = datetime.datetime(mydata[0], mydata[1], mydata[2], mytime[0], mytime[1])
    minute = mytime[1]+mytime[0]*60
    # return int((dt-baseData).__str__().split(' ')[0]),miao,dt.weekday(),round(miao/900)
    isHoliday = 0
    if dt.weekday()in [5,6] or int((dt-baseData).__str__().split(' ')[0]) in [29,28]:
        isHoliday=1
    return isHoliday,minute,int((dt-baseData).__str__().split(' ')[0])

#模型之间的融合，粗暴的取了最值，这个可以再提升
def add2result(result1,result2):
    for each in result2:
        if each in result1:
            #result1[each] = min(result1[each] ,result2[each] )
            result1[each] = result1[each] *0.9 + result2[each] 
        else:
            result1[each] = result2[each]
    return result1



In [ ]:
# 其实就是knn算法，结合了leak。一般的knn+leak应该是0.26分。这里主要有两点创新。一是给算出来的距离值除以频度的1.1次方，
# 这个加了很多分，二是对于新用户又使用了一个新的knn，其他算法在处理新用户的时候也可以参考下。
# knn算法产生的特征可以融合进xgb再训练，已实现，但内存不够弃赛
def training(trainfile = 'train.csv',testfile = 'test.csv',subfile = 'submission.csv' ,
             leak1 = 0.01 ,leak2 = 4 ,leak3 = 20,              #leak
             qidianquan = 10,shijianquan = 10,jiejiaquan = 2,bikequan = 0.5,  #都是拼音，字面意思，越大则这个特征比重越大
             zhishu = 1.06  #对结果影响很大
             ):
    tr = csv.DictReader(open(trainfile))
    # tr 为一个DictReader类，是一个打开的文件流
    #利用train.csv建立user_habit_dict和start_end_dict
    for rec in tr:     #rec为每一行
        user = rec['userid']
        start = rec['geohashed_start_loc']
        end = rec['geohashed_end_loc']
        rec['isHoliday'] , rec['minute'] , rec['data'] = produceTimeInfo(rec['starttime'])
        if user in user_habit_dict: #若用户有记录 则直接添加本行
            user_habit_dict[user].append(rec)
        else:
            user_habit_dict[user] = [rec]

        if start in start_end_dict: #若起点有记录 则直接添加
            start_end_dict[start].append(rec)
        else:
            start_end_dict[start] = [rec]

        if end in end_start_dict:
            end_start_dict[end].append(rec)
        else:
            end_start_dict[end] = [rec]

    print('train done!')
    
    # te是测试文件
    te = csv.DictReader(open(testfile))
    for rec in te:
        user = rec['userid']
        bike = rec['bikeid']
        rec['isHoliday'], rec['minute'], rec['data'] = produceTimeInfo(rec['starttime'])
        if user in user_habit_dict_test:
            user_habit_dict_test[user].append(rec)
        else:
            user_habit_dict_test[user] = [rec]

        if bike in bike_dict:
            bike_dict[bike].append(rec)
        else:
            bike_dict[bike] = [rec]

    print("test done!")


    #sub是提交文件
    sub = open(subfile, 'w')

    iter1 = 0
    # AllhotLocSort = sorted(end_start_dict.items(), key=lambda d: len(d[1]), reverse=True)
    te1 = csv.DictReader(open(testfile))
    for rec in te1:
        iter1 += 1
        if iter1 % 10000 == 0:
            print(iter1/20000,'%',sep='') #sep是实现分隔符，比如多个参数输出时想要输出中间的分隔字符
        # testTime = timeSlipt(rec['minute'])
        rec['isHoliday'], rec['minute'], rec['data'] = produceTimeInfo(rec['starttime'])
        user1 = rec['userid']
        bikeid1 = rec['bikeid']
        order1 = rec['orderid']
        start1 = rec['geohashed_start_loc']
        hour1 = rec['minute']/60
        minute1 = rec['minute']
        isHoliday1 = rec['isHoliday']
        biketype1 = rec['biketype']
        data1 = rec['data']
        result = {}
        hotLoc = {}

        #knn
        if user1 in user_habit_dict:
            for eachAct in user_habit_dict[user1]: #遍历有记录的用户所有记录

                start2 = eachAct['geohashed_start_loc']
                end2 = eachAct['geohashed_end_loc']
                hour2 = eachAct['minute']/60
                isHoliday2 = eachAct['isHoliday']
                biketype2 = eachAct['biketype']
                data2 = rec['data']

                dis = loc_2_dis(start1, start2)  # 全部计算和当前项和test的差异
                dis = min(dis, 1000)    #1000
                qidian= qidianquan * (dis / 100) ** 2

                detalaTime = abs(hour2 - hour1) if abs(hour2 - hour1) < 12 else 24 - abs(hour2 - hour1)
                shijian= shijianquan * (detalaTime / 12 * 10) ** 2

                dayType = isHoliday2 - isHoliday1
                jiejia= jiejiaquan * (dayType * 10) ** 2 #

                biType = int(biketype2) - int(biketype1)
                bike= bikequan * (biType * 10) ** 2  #0.5



                #利用终点预测
                # return 欧式距离,南北方向距离,东西方向距离,曼哈顿距离,方向(-0.5:0.5)
                # test2train_dis = loc_2_dis(start1,end2)
                # train2train_dis = loc_2_dis(start2,end2)
                # dis_detal = min(abs(test2train_dis[3]-train2train_dis[3]),1000)  #1000
                # direction_detal = abs(test2train_dis[4]-train2train_dis[4])
                # direction_detal = direction_detal if direction_detal<0.5 else 1-direction_detal
                # jvli = 4 * (dis_detal/100)**2
                # fangxiang = 1 * (direction_detal/0.5*10)**2

                score = qidian+shijian+jiejia+bike              #jvli+fangxiang

                # print(qidian,shijian,jiejia,bike,jvli,fangxiang)
                if end2 in hotLoc:
                    hotLoc[end2] += 1
                else:
                    hotLoc[end2] = 1
                #hotLoc ={'wx4xxxx':n,...}
                if end2 in result:
                    if result[end2] > score:
                        result[end2] = score   #因为代表差异所以越小代表越接近
                else:
                    result[end2] = score
                #result = {'wx4xxxx':score,....}  #即当前test条目下，根据当前记录的项的终点的差异
            for each in hotLoc:
                result[each] = result[each] / (hotLoc[each]**zhishu)  #0
            #出现次数越多，score越小，即可信度越高
            for each in result:
                result[each] = math.sqrt(result[each]) #开个方 不知道有啥意义
        print(result)
        #利用test中的用户历史记录
        if user1 in user_habit_dict_test:
            resulttest = {}
            user_habit_dict_test[user1].sort(key = lambda x:x['data']*60*24+x['minute'])
            xuhao = 0
            for i in range(len(user_habit_dict_test[user1])-1):
                if user_habit_dict_test[user1][i]['orderid'] == order1:
                    xuhao = i
                    resulttest[user_habit_dict_test[user1][i+1]['geohashed_start_loc']] = 21
            for i in range(len(user_habit_dict_test[user1])):
                if i not in [xuhao,xuhao+1]:
                    resulttest[user_habit_dict_test[user1][i]['geohashed_start_loc']] = 21+abs(i-xuhao)
                result = add2result(result, resulttest)

        # leak
        '''
        if bikeid1 in bike_dict:
            resultleak = {}
            bike_dict[bikeid1].sort(key = lambda x:x['data']*60*24+x['minute'])
            for i in range(len(bike_dict[bikeid1])-1):
                if bike_dict[bikeid1][i]['orderid'] == order1:
                    zhong = bike_dict[bikeid1][i+1]['data']*60*24+bike_dict[bikeid1][i+1]['minute']
                    qi = bike_dict[bikeid1][i]['data']*60*24+bike_dict[bikeid1][i]['minute']
                    detal = zhong-qi
                    if detal<30:
                        resultleak[bike_dict[bikeid1][i + 1]['geohashed_start_loc']] = leak1

                    elif detal<2*60:
                        resultleak[bike_dict[bikeid1][i + 1]['geohashed_start_loc']] = leak2  #4

                    else:
                        resultleak[bike_dict[bikeid1][i + 1]['geohashed_start_loc']] = leak3   #20
            result = add2result(result,resultleak)
        '''

        #起点终点对的knn
        if start1 in start_end_dict:
            endDict = {}
            resultqizhong={}
            for eachAct in start_end_dict[start1]:
                score = 0
                score += (24-abs(hour1-eachAct['minute']/60))/24
                score += (1-abs(isHoliday1-eachAct['isHoliday']))*0.4
                if eachAct['geohashed_end_loc'] in endDict:
                    endDict[eachAct['geohashed_end_loc']] += score
                else:
                    endDict[eachAct['geohashed_end_loc']] = score
            hotLoc = sorted(endDict.items(),key = lambda x:x[1],reverse=True)
            if len(hotLoc)>=1:
                resultqizhong[hotLoc[0][0]] = 1000
            if len(hotLoc) >= 2:
                resultqizhong[hotLoc[1][0]] = 1001
            if len(hotLoc) >= 3:
                resultqizhong[hotLoc[2][0]] = 1002
            result = add2result(result, resultqizhong)

        #剔除不合理结果
        for each in result:
            distance = loc_2_dis(each,start1)
            if distance > 2500:
                result[each] = 1999


        if start1 in result:
            result[start1] = min(2000, result[start1])
        else:
            result[start1]=2000
        result['fuck2'] = 2001
        result['fuck3'] = 2002

        bestResult = sorted(result.items(), key=lambda d: d[1])
        string = rec['orderid']
        num = 0
        for item in bestResult:
            #print(item)
            string += ',' + item[0]
            #print(item[0])
            #print('---------')
            # string += ':' + str(item[1]) + '\t'
            num += 1
            if num == 3:
                break
        sub.write(string + '\n')

    sub.close()
    print('ok')



In [ ]:
training(r'F:\data\mobike\train.csv', r'F:\data\mobike\test.csv', r'F:\data\mobike\submission.csv' )

train done!
test done!
{'wx4gg04': 6.863956830498947, 'wx4g6zb': 15.408362758845826, 'wx4g6xy': 17.858345171627832, 'wx4g6z1': 17.95368378729303, 'wx4g6xe': 17.701868478265485}
{'wx4eqfx': 0.0, 'wx4eu36': 8.884044920590176, 'wx4eqep': 13.979625293326185}
{'wx4e5ye': 1.70701310980226, 'wx4e5yg': 6.3801059338284425}
{'wx4equ4': 16.001700905432195, 'wx4dxtx': 4.1615641703377415, 'wx4eqsu': 16.11214170948095}
{}
{'wx4ew9t': 17.986993191395783, 'wx4ew9s': 13.780326701995753, 'wx4ewd9': 7.729659204727808}
{'wx4ekbw': 2.0626900208895775, 'wx4ekcs': 6.169151361209598, 'wx4ekc1': 5.84705830462633}
{'wx4f8mn': 5.756723307608205}
{'wx4ffy2': 14.84126942116942, 'wx4g2nw': 15.23351466061543, 'wx4g2np': 16.039483517106596, 'wx4st8m': 9.885430164607694}
{'wx4engu': 17.686634854451178}
{'wx4er6q': 17.445736271103353, 'wx4erd2': 14.398456052341654}
{}
{'wx4ff61': 13.480222502278, 'wx4g5fd': 0.24330820434694753, 'wx4g5gm': 8.414315281588499, 'wx4ejmn': 13.84176850705229, 'wx4ff70': 15.558053235519179}
{

{'wx4f3hy': 12.861378898462867, 'wx4f7d2': 16.131251323046342, 'wx4f3jy': 13.133118249025545, 'wx4f791': 13.173751469736539, 'wx4f3k3': 9.0502472414748, 'wx4f3m1': 15.740870220112354}
{'wx4g60c': 6.945129873012671, 'wx4g39j': 0.6999407255977435}
{'wx4ffdn': 16.361362815146787, 'wx4ffd2': 14.457903659349236, 'wx4ff92': 0.9170847702308016, 'wx4ffdf': 3.931569493545031, 'wx4ffdj': 14.362760974431563}
{'wx4gd7m': 16.175319806440562, 'wx4gd7j': 8.350838720950822, 'wx4gd7c': 9.893396830751298, 'wx4gd99': 8.454502931110168}
{'wx4endm': 14.174351812470901, 'wx4ejc2': 4.069920055996805}
{'wx4dxyq': 12.636382799210407, 'wx4fdrb': 6.738701394647353, 'wx4f8j9': 13.482405100870553, 'wx4ff0t': 17.913212418736123, 'wx4fdpy': 4.317923552815481, 'wx4ghc2': 18.468535287191475, 'wx4ff14': 6.842829352518771, 'wx4fdpz': 0.0777711917330823}
{}
{'wx4e5sw': 1.6848713653814562, 'wx4g5be': 7.580138534082632}
{'wx4fhh9': 1.9575200286930392, 'wx4fhj3': 16.78661289984972, 'wx4fhj8': 9.221449939297036, 'wx4fhjn': 1

{'wx4gd4m': 15.485509708338363, 'wx4g919': 15.627099809770748}
{'wx4dw22': 15.982425435357946, 'wx4dq61': 14.64689370227436}
{'wx4ep93': 14.769723378643398, 'wx4dxd0': 6.034834086399475, 'wx4ep3v': 5.898783943309846, 'wx4ep91': 19.24821882695468, 'wx4ep92': 6.629135185461145}
{'wx4g9py': 15.455859840072781, 'wx4fbbf': 5.222871159014813}
{}
{'wx4drz4': 8.786545186695216, 'wx4f2ws': 17.163964037394457, 'wx4dryb': 2.8409428273935085}
{'wx4dwhe': 10.30031244336297, 'wx4dwhr': 16.23446881702839, 'wx4dwj4': 20.163464812403117, 'wx4dwhc': 7.6808062361472995, 'wx4dwhq': 15.106514727985886}
{'wx4gjsj': 3.014150268748242, 'wx4g3bm': 13.835123907992942, 'wx4ujqr': 14.716835290428598, 'wx4gjev': 3.017924890298082, 'wx4gjk3': 11.649521002432047}
{'wx4expp': 12.649368451078244, 'wx4exr2': 11.984962924101655, 'wx4exqy': 12.760318599339751}
{}
{}
{}
{'wx4g4cf': 12.07725822272207}
{'wx4dysg': 10.979862886180323, 'wx4dyt9': 10.865847480265213, 'wx4dy32': 15.502567445324397}
{}
{'wx4g4vg': 8.868394562564

### 用dict保存信息
#user_habit_dict:每个用户的乘车记录:起点,终点,距离
user_habit_dict={}

#start_end_dict:每条记录的起点,终点对
start_end_dict={}

#end_start_dict:每条记录的起点,终点对
end_start_dict={}

#user_habit_dict_test:test中每个用户的记录
user_habit_dict_test={}

#bike_dict:bike中的记录
bike_dict={}


In [12]:
from random import choice
#print(user_habit_dict[choice(list(user_habit_dict.keys()))]) #随即打印一条看看
print(start_end_dict[choice(list(start_end_dict.keys()))]) #随即打印一条看看

[{'geohashed_start_loc': 'wx4dnu8', 'isHoliday': 0, 'biketype': '1', 'bikeid': '329100', 'minute': 1161, 'orderid': '3162575', 'data': 10, 'starttime': '2017/5/11 19:21', 'userid': '1027944', 'geohashed_end_loc': 'wx4dng7'}, {'geohashed_start_loc': 'wx4dnu8', 'isHoliday': 0, 'biketype': '1', 'bikeid': '329100', 'minute': 1161, 'orderid': '3162575', 'data': 10, 'starttime': '2017/5/11 19:21', 'userid': '1027944', 'geohashed_end_loc': 'wx4dng7'}, {'geohashed_start_loc': 'wx4dnu8', 'isHoliday': 0, 'biketype': '1', 'bikeid': '329100', 'minute': 1161, 'orderid': '3162575', 'data': 10, 'starttime': '2017/5/11 19:21', 'userid': '1027944', 'geohashed_end_loc': 'wx4dng7'}]
